# This is a example to test if our padding algorithm works

## Overview: 

Padding is to solve the problem that the finemapping regions between GWAS and xqtl are not the same. For GWAS the finemapping regions are usually LD blocks, but for xqtl it's designed to be on TAD/TADB/cis window/ extended cis window etc. Here based on the nature that LD blocks is considered as independent from each other, we can paste the result (lbf matrix) multiple LD blocks to match the region of our xqtl finemapping. For those columns that are not shared between qtl and GWAS data, we will remove these columns.

To paste out a new large matrix for GWAS lbf, our strategy is: if pasting involves 2 LD blocks for example, we will fill up the left upper part with lbf matrix from LD1, then the lower right part with lbf matrix from LD2, the rest of them we will fill up with 0. 

Then after we cook up the new lbf matrices both for qtl and GWAS, we need to re-compute the alpha matrix and PIP. For eqtl credible sets may require do finemapping again. for now we choose a simple strategy: just remove the variant in any credible sets if they are not shared by the two(or more) datasets.

## Input 1: GWAS finemapping result folder

Here I used our ADHD GWAS finemapping result: `/home/hs3393/ADHD/ADHD_finemap`

## Input 2: qtl finemapping result 

Better use `ls /mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/susie_per_gene_tad/cache/*rds` as input, so that we can create job number = gene number.

This is the eqtl finemapping result by Hao on AD.

## Step details:

In [1]:
library(tidyverse)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.1     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


In [2]:
# read all GWAS finemapping result file name to extract the chr, start, end and corresponding file paths  

gwas_finemap_result = list.files("/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap", full.names=T, pattern = "\\.rds$")

gwas_file_tb = tibble(file_path = gwas_finemap_result)

# use regex to extract the start, end and chromosome of LD blocks from the file name
match_pattern = function(filename){
    pattern <- "chr[0-9XY]+_(\\d+_\\d+)"
    result = regmatches(filename, regexpr(pattern, filename))
    return(result)
}
  
LD_block_position = gwas_file_tb %>% mutate(chr_pos = map(file_path, match_pattern)) %>% 
  separate(chr_pos, into = c("chr", "start", "end")) %>% relocate(file_path, .after = end) %>%
    mutate(start = as.numeric(start), end = as.numeric(end)) %>% arrange(chr, start)

In [3]:
head(LD_block_position)

chr,start,end,file_path
<chr>,<dbl>,<dbl>,<chr>
chr1,16103,2888443,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr1.chr1_16103_2888443.unisusie_rss.fit.rds
chr1,2888443,4320284,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr1.chr1_2888443_4320284.unisusie_rss.fit.rds
chr1,4320284,5853833,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr1.chr1_4320284_5853833.unisusie_rss.fit.rds
chr1,5853833,7110219,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr1.chr1_5853833_7110219.unisusie_rss.fit.rds
chr1,7110219,9473386,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr1.chr1_7110219_9473386.unisusie_rss.fit.rds
chr1,9473386,11328222,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr1.chr1_9473386_11328222.unisusie_rss.fit.rds


# Part 2: input - qtl finemapping result files (rds format)

In [4]:
qtl_list = list.files("/mnt/vast/hpc/csg/molecular_phenotype_calling/eqtl/output/susie_per_gene_tad/cache", full.names=T, pattern = "\\.rds$")
qtl_file = readRDS(qtl_list[100])

In [5]:
qtl_chr = regmatches(qtl_file$dlpfc_eqtl$variable_name[1], regexpr("(chr[0-9]+)", qtl_file$dlpfc_eqtl$variable_name[1]))
qtl_start  <- as.numeric(sub(".*:(\\d+)_.*", "\\1", qtl_file$dlpfc_eqtl$variable_name[1]))
qtl_end  <- as.numeric(sub(".*:(\\d+)_.*", "\\1", qtl_file$dlpfc_eqtl$variable_name[length(qtl_file$dlpfc_eqtl$variable_name)]))

In [6]:
qtl_chr
qtl_start 
qtl_end 

[1] "chr7"

[1] 76680351

[1] 80078654

In [7]:
# retract related files in GWAS
related_LD = LD_block_position %>% filter(chr == qtl_chr) %>% filter((start <= qtl_start & end >= qtl_start) |
                                         (start >= qtl_start & end <= qtl_end) | 
                                         (start <= qtl_end & end >= qtl_end))

In [8]:
related_LD

chr,start,end,file_path
<chr>,<dbl>,<dbl>,<chr>
chr7,73724576,77106024,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr7.chr7_73724576_77106024.unisusie_rss.fit.rds
chr7,77106024,78547096,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr7.chr7_77106024_78547096.unisusie_rss.fit.rds
chr7,78547096,80918333,/mnt/mfs/hgrcgrid/homes/hs3393/ADHD/ADHD_finemap/ADHD_sumstat_hg38_qc.chr7.chr7_78547096_80918333.unisusie_rss.fit.rds


## Step 3: cook up new lbf matrix

In [9]:
# extract those related GWAS finemapping result lbf matrix to form the larger one
cnt = 1
variants = c()
lbf_mtx = list()
for (file in related_LD$file_path){
    rds = readRDS(file)
    variants = c(variants, rds$variants)
    lbf_mtx[[cnt]] = as.data.frame(rds$lbf_variable)
    colnames(lbf_mtx[[cnt]]) = rds$variants
    cnt = cnt + 1
}

# after combining the matrices, fill those NA with 0 to form the whole matrix
lbf_whole_mtx = bind_rows(lbf_mtx) %>% replace(is.na(.), 0)

# get the shared variants between gwas and qtl

# here is one problem: maybe now the output of finemapping does not need $dlpfc... and the variant name have different format
# chr:9999 or chr_999, so here we change : to _; this brings a lot of trouble
# if now variable name are uniformed, then things are good, remove the str_replcace line
  
shared_variant = unlist(intersect(qtl_file$dlpfc_eqtl$variable_name %>% 
  map(~ str_replace_all(.x, ":", "_")), colnames(lbf_whole_mtx)))

  
# remove those columns that does not share SNP, only keep those shared by two dataset
GWAS_lbf_matrix = lbf_whole_mtx[, shared_variant]
  

# again, change the variant names
colnames(qtl_file$dlpfc_eqtl$lbf_variable) = unlist(qtl_file$dlpfc_eqtl$variable_name %>% 
  map(~ str_replace_all(.x, ":", "_")))

# because in the susie output the cs are recorded by index, so we get the index that are removed
rm_index = which(!(colnames(qtl_file$dlpfc_eqtl$lbf_variable) %in% shared_variant))
  
# also, for qtl data, only keep variants that are shared
qtl_lbf_mtx = qtl_file$dlpfc_eqtl$lbf_variable[,shared_variant]

## Step 4: based on lbf matrix to get new alpha, PIP and cs

In [11]:
# convert lbf to alpha
lbf_to_alpha_vector = function(lbf, prior_weights = NULL) {
  if (is.null(prior_weights)) prior_weights = 1/length(lbf)
  maxlbf = max(lbf)
  # w is proportional to BF, but subtract max for numerical stability.
  w = exp(lbf - maxlbf)
  # Posterior prob for each SNP.
  w_weighted = w * prior_weights
  weighted_sum_w = sum(w_weighted)
  alpha = w_weighted / weighted_sum_w
  return(alpha = alpha)
}

lbf_to_alpha = function(lbf) t(apply(lbf, 1, lbf_to_alpha_vector))

# convert lbf to pip
lbf_to_pip = function(lbf) {
    alpha = lbf_to_alpha(lbf)
    return(as.vector(1 - apply(1 - alpha,2,prod)))
}

new_mu = function(susie_obj,shared_variant,V=1,residual_variance =1){
    new_X <- susie_obj$input_data$X_resid[,str_replace(shared_variant, "_", ":")]
    XtX <- t(new_X) %*% new_X
    dXtX <- diag(XtX)
    Xty <- t(new_X) %*% (as.numeric(susie_obj$input_data$Y_resid))
    post_var = (1/V + dXtX/residual_variance)^(-1) # Posterior variance.
    post_mean = (1/residual_variance) * post_var * Xty
}

# use new lbf matrix to compute alpha and pip
GWAS_alpha = lbf_to_alpha(GWAS_lbf_matrix)
GWAS_pip = lbf_to_pip(GWAS_lbf_matrix)

qtl_alpha = lbf_to_alpha(qtl_lbf_mtx)
qtl_pip = lbf_to_pip(qtl_lbf_mtx)
  
qtl_mu= new_mu(qtl_file$dlpfc_eqtl,shared_variant)

# for now, we remove the variants in cs, if that variant is not shared by two traits
## note: this part may be changed!! now is just a rough strategy
cs_number = length(qtl_file$dlpfc_eqtl$sets$cs)
new_cs = list()
if(cs_number == 0){
    new_cs = NA
}else{
    for (i in (1:cs_number)){
     new_cs[[i]] = setdiff(qtl_file$dlpfc_eqtl$sets$cs[[i]], rm_index)
    }
}

# the output can be: GWAS+ qtl lbf matrix; alpha; pip
# and qtl new cs

In [12]:
str(qtl_pip)

 num [1:10384] 0.000905 0.00082 0.000818 0.000915 0.000815 ...


In [13]:
str(GWAS_pip)

 num [1:10384] 0.00289 0.00289 0.00289 0.00289 0.00289 ...


Now they share same variants and variant number, can do susie_coloc!

In [14]:
str(GWAS_alpha)

 num [1:30, 1:10384] 9.63e-05 9.63e-05 9.63e-05 9.63e-05 9.63e-05 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:10384] "chr7_76680351_T_C" "chr7_76680425_A_C" "chr7_76681272_G_GA" "chr7_76681467_T_C" ...


In [15]:
str(qtl_alpha)

 num [1:10, 1:10384] 9.16e-05 9.16e-05 9.13e-05 9.07e-05 9.02e-05 ...
 - attr(*, "dimnames")=List of 2
  ..$ : NULL
  ..$ : chr [1:10384] "chr7_76680351_T_C" "chr7_76680425_A_C" "chr7_76681272_G_GA" "chr7_76681467_T_C" ...


In [16]:
qtl_alpha   


chr7_76680351_T_C,chr7_76680425_A_C,chr7_76681272_G_GA,chr7_76681467_T_C,chr7_76681738_T_G,chr7_76682203_C_T,chr7_76683495_T_TTA,chr7_76683617_A_G,chr7_76684943_C_T,chr7_76685024_G_A,⋯,chr7_80076814_G_A,chr7_80077042_A_G,chr7_80077068_GC_G,chr7_80077606_T_C,chr7_80077672_A_T,chr7_80077844_C_T,chr7_80077915_GAAA_G,chr7_80077915_GAAA_GA,chr7_80078071_G_C,chr7_80078654_C_T
9.164570e-05,8.424680e-05,8.405532e-05,9.254683e-05,8.378092e-05,9.455289e-05,8.365033e-05,8.705025e-05,8.803159e-05,8.362935e-05,⋯,8.721797e-05,8.721797e-05,8.688558e-05,8.721797e-05,8.721797e-05,8.721797e-05,8.779005e-05,8.709950e-05,8.622527e-05,8.721797e-05
9.162835e-05,8.421167e-05,8.401974e-05,9.253199e-05,8.374470e-05,9.454316e-05,8.361381e-05,8.702185e-05,8.800548e-05,8.359277e-05,⋯,8.718995e-05,8.718995e-05,8.685679e-05,8.718995e-05,8.718995e-05,8.718995e-05,8.776265e-05,8.707119e-05,8.619429e-05,8.718995e-05
9.126114e-05,8.348064e-05,8.327937e-05,9.221651e-05,8.299123e-05,9.433301e-05,8.285407e-05,8.642797e-05,8.745889e-05,8.283192e-05,⋯,8.660544e-05,8.660544e-05,8.625626e-05,8.660544e-05,8.660544e-05,8.660544e-05,8.718941e-05,8.648061e-05,8.554772e-05,8.660544e-05
9.071770e-05,8.243327e-05,8.221906e-05,9.174613e-05,8.191284e-05,9.401015e-05,8.176703e-05,8.557033e-05,8.666664e-05,8.174331e-05,⋯,8.576185e-05,8.576185e-05,8.539051e-05,8.576185e-05,8.576185e-05,8.576185e-05,8.635735e-05,8.562855e-05,8.461477e-05,8.576185e-05
9.018283e-05,8.143853e-05,8.121248e-05,9.127951e-05,8.088981e-05,9.367986e-05,8.073614e-05,8.474854e-05,8.590437e-05,8.071099e-05,⋯,8.495397e-05,8.495397e-05,8.456241e-05,8.495397e-05,8.495397e-05,8.495397e-05,8.555548e-05,8.481284e-05,8.372160e-05,8.495397e-05
8.979576e-05,8.073881e-05,8.050470e-05,9.093980e-05,8.017088e-05,9.343378e-05,8.001188e-05,8.416632e-05,8.536254e-05,7.998574e-05,⋯,8.438180e-05,8.438180e-05,8.397651e-05,8.438180e-05,8.438180e-05,8.438180e-05,8.498474e-05,8.423531e-05,8.308925e-05,8.438180e-05
8.962801e-05,8.044038e-05,8.020290e-05,9.079210e-05,7.986442e-05,9.332545e-05,7.970320e-05,8.391702e-05,8.513008e-05,7.967665e-05,⋯,8.413679e-05,8.413679e-05,8.372576e-05,8.413679e-05,8.413679e-05,8.413679e-05,8.473963e-05,8.398804e-05,8.281853e-05,8.413679e-05
8.969065e-05,8.055143e-05,8.031520e-05,9.084728e-05,7.997845e-05,9.336604e-05,7.981804e-05,8.400988e-05,8.521668e-05,7.979165e-05,⋯,8.422801e-05,8.422801e-05,8.381911e-05,8.422801e-05,8.422801e-05,8.422801e-05,8.483094e-05,8.408010e-05,8.291933e-05,8.422801e-05
8.995510e-05,8.102527e-05,8.079444e-05,9.107966e-05,8.046515e-05,9.353562e-05,8.030831e-05,8.440493e-05,8.558474e-05,8.028258e-05,⋯,8.461646e-05,8.461646e-05,8.421673e-05,8.461646e-05,8.461646e-05,8.461646e-05,8.521904e-05,8.447215e-05,8.334852e-05,8.461646e-05
9.037393e-05,8.179174e-05,8.156985e-05,9.144598e-05,8.125297e-05,9.379856e-05,8.110207e-05,8.504045e-05,8.617540e-05,8.107744e-05,⋯,8.524178e-05,8.524178e-05,8.485727e-05,8.524178e-05,8.524178e-05,8.524178e-05,8.584147e-05,8.510340e-05,8.403952e-05,8.524178e-05


Gwas matrix have 30 rows, which correspond to our design. And they have same column number.

In [17]:
str(new_cs)

 logi NA


In [18]:
str(qtl_mu)

 num [1:10384, 1] 0.03118 0.00972 0.00827 0.03327 0.00569 ...
 - attr(*, "dimnames")=List of 2
  ..$ : chr [1:10384] "chr7:76680351_T_C" "chr7:76680425_A_C" "chr7:76681272_G_GA" "chr7:76681467_T_C" ...
  ..$ : NULL
